# ДОВЕРИТЕЛЬНЫЙ ИНТЕРВАЛ ДЛЯ СРЕДНЕГО ПРИ ИЗВЕСТНОМ ИСТИННОМ СТАНДАРТНОМ ОТКЛОНЕНИИ

In [2]:
# Импорт библиотек
from pathlib import Path
from warnings import filterwarnings
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Скрывает INFO и WARNING, оставляет только ERROR
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0' # Отключить oneDNN сообщения

# Автоматическое определение пути к текущему окружению conda
conda_prefix = os.environ.get('CONDA_PREFIX')
if conda_prefix:
    os.environ['XLA_FLAGS'] = f'--xla_gpu_cuda_data_dir={conda_prefix}'

# --- 2. Включение ускорения Intel для CPU --- scikit-learn
try:
    from sklearnex import patch_sklearn, config_context
    patch_sklearn()
    print('Intel Extension для scikit-learn успешно активирован')
except (ImportError, ModuleNotFoundError, Exception) as e:
    print(f'Не удалось активировать Intel Extension для scikit-learn: {e}')
    print(f'Продолжаем работу без ускорения (это нормально)')


from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#from arch import arch_model
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy
from surprise import BaselineOnly

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd
from scipy.optimize import minimize, least_squares


from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import sklearn
import math

import time

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model


# Если есть TensorFlow, проверяем его:
try:
    import tensorflow as tf
    print(f"TensorFlow видит GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")
except ImportError:
    pass



Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


Intel Extension для scikit-learn успешно активирован


2025-12-16 20:44:48.247826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-16 20:44:48.267555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-16 20:44:48.274754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow видит GPU: True


I0000 00:00:1765907093.601938   20218 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765907093.772688   20218 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765907093.772776   20218 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
filterwarnings("ignore")
# warnings.filterwarnings('ignore', category=UserWarning)
# warnings.filterwarnings('ignore', message='.*ConvergenceWarning.*')

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")
# %matplotlib inline

SEED = 42

# Проверка текущей рабочей директории
print(f"Текущая рабочая директория: {os.getcwd()}")

# Текущая рабочая директория (скорее всего .../IDE)
current_dir = os.getcwd()

# Относительный путь от корня проекта до папки с ноутбуком
# Обратите внимание: используем r'' для корректной обработки слешей и пробелов
relative_path_to_notebook = r'skillfactory/DS_PROD-4'

# Проверяем, не перешли ли мы уже в нужную папку (чтобы не было ошибок при перезапуске ячейки)
if not current_dir.endswith("DS_PROD-4"):
    # Собираем полный путь
    new_dir = os.path.join(current_dir, relative_path_to_notebook)
    
    # Меняем рабочую директорию
    try:
        os.chdir(new_dir)
        print(f"Рабочая директория изменена на: {os.getcwd()}")
    except FileNotFoundError:
        print("Ошибка: путь не найден. Проверьте правильность названий папок.")
else:
    print("Рабочая директория уже верная.")

Текущая рабочая директория: /home/pavel/IDE/skillfactory/DS_PROD-4
Рабочая директория уже верная.


Пора нам познакомиться с формулой доверительного интервала для истинного среднего при известном стандартном отклонении: 

 $\mu=X_{mean}\pm z_{крит} \times \frac{\sigma}{\sqrt{n}}$

 Таким образом, 
 $z_{крит}$ — это значение, которое отсекает критическую область нормального распределения. 

Это константа, которая берётся из таблицы, и её значение мы задаём, когда указываем надёжность, с которой хотим построить интервал:

$z_{крит}=z_{\frac{1-\gamma}{2}} = z_{\frac{\alpha}{2'}}$

где $\gamma$ — заданный уровень надёжности (доверия);

$\alpha = 1 - \gamma$ — уровень значимости.

Тогда

$$\mu = X_{mean} \pm z_{\frac{\alpha}{2}} \times \frac{\sigma}{\sqrt{n}}$$



Была опрошена случайная выборка из 36 жителей региона об их затратах на продукты питания за последний месяц. Выборочное среднее оказалось равным 16 100 рублей.
Допустим, откуда-то известно, что истинное стандартное отклонение расходов равно 12 000 рублей.
Нам нужно построить доверительный интервал для истинных средних расходов жителя данного региона на продукты питания в месяц. Интервал должен определять истинное значение с надежностью в 95 %.

In [4]:
n = 36 # размер выборки
x_mean = 16100 # выборочное среднее
sigma = 12000 # истинное стандартное отклонение
gamma = 0.95 # уровень надёжности
alpha = 1 - gamma # уровень значимости



Найти значение z-критического для любого уровня значимости можно при помощи функции norm.ppf() из модуля scipy.stats. По специфическим причинам реализации функция возвращает не совсем z-критическое, а его отрицательный модуль, поэтому мы берём его со знаком минус.

Функция norm.ppf(x) возвращает значение z-крит, левее (ниже) которого лежит доля  левого хвоста стандартного нормального распределения. Выражение norm.ppf(α/2) вернёт $-z_{\frac{\alpha}{2}}$, то есть значение  $z_{крит}$, ниже которого $\frac{\alpha}{2}$ всех значений стандартного нормального распределения. Это число отрицательное, мы берём его со знаком минус, чтобы получить $z_{\frac{\alpha}{2}}$.

In [5]:
from scipy.stats import norm
z_crit = -norm.ppf(alpha/2) # z-критическое
print(z_crit)
# 1.959963984540054

1.959963984540054


Итак, теперь у нас есть всё, чтобы построить доверительный интервал. Напишем функцию z_mean_conf_interval(), которая будет принимать на вход:

mean — выборочное среднее;

sigma — истинное стандартное отклонение;

n — размер выборки;

gamma — уровень надёжности.

Функция будет вычислять нижнюю и верхнюю границы по формуле и возвращать их в виде кортежа.

Округлим результат до целого и выведем на экран:

In [6]:
def z_mean_conf_interval(x_mean, sigma, n, gamma=0.95):  
    alpha = 1 - gamma  # уровень значимости
    z_crit = -norm.ppf(alpha/2)  # z-критическое
    eps = z_crit * sigma/(n ** 0.5) # погрешность
    lower_bound = x_mean - eps  # левая (нижняя) граница
    upper_bound = x_mean + eps  # правая (верхняя) граница
    # возвращаем кортеж из границ интервала
    return lower_bound, upper_bound

# строим доверительный интервал для среднего при известном СКО
lower_bound, upper_bound = z_mean_conf_interval(x_mean, sigma, n)

# выводим результат
print('Доверительный интервал: {}'.format((round(lower_bound), round(upper_bound))))

# Доверительный интервал: (12180, 20020)

Доверительный интервал: (12180, 20020)


Примечание. Заметьте, что это универсальный код. Если перед вами возникнет другая задача расчёта доверительного интервала для истинного среднего, то вам достаточно будет правильно поменять значения переменных.

Полученный результат интерпретируется следующим образом: мы на 95 % уверены, что истинные расходы жителей на продукты питания лежат в интервале от 12 180 рублей до 20 020 рублей. 


⚡ Важно! Доверительный интервал не даёт нам 100 % результата. Всегда есть шанс, что мы промахнулись и не захватили истинного значения. Этот шанс как раз равен уровню значимости .
То есть в нашем примере существует 5 %-ый шанс, что истинное среднее расходов вовсе не лежит в интервале от 12 180 до 20 020. Существует вероятность, что математическое ожидание на самом деле равно 5 000 рублей или 25 000 рублей. Но такая вероятность составляет менее 5 %, и это нас устраивает.

К сожалению, нельзя построить доверительный интервал, охватывающий 100 % возможных значений, так как границы такого доверительного интервала $-\infty$ и $\infty$. Как мы знаем, это абстрактные математические понятия, неприменимые на практике.

### Задание 6.5

3 points possible (graded)

Аккумуляторный завод производит батарейки, срок службы которых имеет истинное стандартное отклонение в 2.4 месяца. Средний срок службы батареек в случайной выборке из 64 штук составил 12.35 месяца. Необходимо построить 90 %-ный доверительный интервал для истинного среднего срока службы производимой батарейки и ответить на вопросы заданий ниже.

Чему равно значение z-критического для такого интервала? Введите число с двумя знаками после разделительного знака.


Теперь постройте 90 %-ный доверительный интервал. Используйте критическое значение, округлённое до двух знаков. Ответ введите в виде нижней и верхней границы интервала, округляя результат до двух знаков после разделительной точки.

Нижняя граница:


Верхняя граница:




Подсказка (1 из 2): z-критическое можно определить с помощью функции norm.ppf(alpha/2), умножив результат на -1. alpha — уровень значимости (1-gamma = 1-0.9 = 0.1).
Подсказка (2 из 2): Вычислив z-критическое, вам останется только подставить размер выборк (n=64), выборочное среднее (x_mean = 12.35) и стандартное отклонение (sigma=2.4) в формулу доверительного интервала для истинного среднего при известном стандартном отклонении, которую мы приводили в модуле выше.

In [11]:
n = 64 # размер выборки
x_mean = 12.35 # выборочное среднее
sigma = 2.4 # истинное стандартное отклонение
gamma = 0.90 # уровень надёжности
alpha = 1 - gamma # уровень значимости

z_crit = -norm.ppf(alpha/2) # z-критическое
print(round(z_crit, 2))
#1.64

1.64


In [ ]:
# строим доверительный интервал для среднего при известном СКО
lower_bound, upper_bound = z_mean_conf_interval(x_mean, sigma, n, gamma)

# выводим результат
print('Доверительный интервал: {}'.format((round(lower_bound, 2), round(upper_bound, 2))))
# Доверительный интервал: (11.86, 12.84)

Доверительный интервал: (11.86, 12.84)


## ДОВЕРИТЕЛЬНЫЙ ИНТЕРВАЛ ДЛЯ СРЕДНЕГО ПРИ НЕИЗВЕСТНОМ СТАНДАРТНОМ ОТКЛОНЕНИИ

Внимательный студент заметил, что в формуле доверительного интервала для истинного среднего мы использовали значение истинного стандартного отклонения $\sigma$.

Для правомерного использования формулы истинное стандартное отклонение выборки должно быть известно. Но, как правило, нам неизвестно это значение (если только оно не свалилось на нас озарением), ведь для этого нам нужно знать все значения генеральной совокупности. А если они у нас есть, то нам и доверительные интервалы не нужны — зная всю генеральную совокупность, всегда можно найти истинные параметры.

В реальной жизни нам приходится заменять значение $\sigma$ значением выборочного стандартного отклонения, которое мы можем посчитать на основании выборки.

Когда мы заменяем константу на случайную величину, точность всей конструкции явно становится меньше. Вероятно, итоговая величина будет похожа на нормальное распределение, но будет обладать бОльшим разбросом.

Такая случайная величина хорошо описывается распределением Стьюдента, или Т-распределением. 

Распределение Стьюдента — это целое семейство родственников стандартного нормального распределения.

Действительно, центр (среднее) по-прежнему будет в 0, а вот разброс значений (то есть стандартное отклонение) будет уже больше. Этот разброс будет различным в зависимости от степеней свободы ($k$).


Степени свободы  обозначаются d.f. (от англ. degrees of freedom). Они и говорят нам о том, насколько велик разброс в значениях распределения Стьюдента. Этот параметр вычисляется как $k = n - 1$, где $n$ — размер выборки.


Тогда в формуле для доверительного интервала случайная нормальная величина $Z$ заменится на случайную величину $T$ , а истинное стандартное отклонение $\sigma$ заменится на выборочное — $X_{std}$:

$$\mu=X_{mean}\pm t_{крит} \times \frac {X_{std}}{\sqrt{n}}$$


Значения $t_{крит}$ вычисляются аналогично $z_{крит}$: задаётся уровень надёжности (или уровень значимости  $\alpha$), и на его основе по таблицам или калькуляторам для распределения Стьюдента вычисляется число. Отличие состоит в том, что для вычисления $t_{крит}$ нужно задать ещё и число степеней свободы $k = n - 1$:

$$t_{крит}(k)=t_{\frac{(1-\gamma)}{2}}(k)=t_{\frac{\alpha}{2}}(k)$$



### Задача

Управляющий супермаркета хочет оценить, сколько денег покупатели тратят за один поход в супермаркет в среднем. На основании ранее собранных данных известно, что расходы одного покупателя распределены приблизительно нормально. Управляющий взял случайную выборку из 15 покупателей и выяснил, что их средние расходы равны 2 000 рублей, а выборочное стандартное отклонение равно 400 рублей.

Для оценки управляющий хочет построить доверительный интервал с надёжностью в 95 %.

Давайте поможем управляющему, решив несколько задачек с помощью Python.

Итак, зададим необходимые для решения задачи переменные:

In [16]:
n = 15 # размер выборки
k = n - 1 # число степеней свободы
x_mean = 2000 # выборочное среднее
x_std = 400 # выборочное стандартное отклонение
gamma = 0.95 # уровень надёжности
alpha = 1 - gamma # уровень значимости

In [17]:
from scipy.stats import t
t_crit = -t.ppf(alpha/2, k) # t-критическое
print(t_crit)
# 2.1447866879169273

2.1447866879169273


Примечание. Интересен тот факт, что значение t-критического при том же уровне надёжности чуть выше, чем значение z-критического, например при уровне надёжности 95 % это значение будет 1.96, а вот для t-критического будет 2.14. Благодаря этому доверительный интервал при неизвестном стандартном отклонении гарантировано будет шире, чем при известном, если $\sigma=X_{std}$. 

Теперь составим доверительный интервал для истинного среднего. Давайте напишем функцию для вычисления доверительного интервала для среднего — t_mean_conf_interval(), модифицировав функцию для z-интервала согласно приведённым выше рассуждениям. Функция всё так же будет возвращать левую и нижнюю границу по формуле.

Округлим результат и выведем на экран:

In [18]:
def t_mean_conf_interval(x_mean, x_std, n, gamma=0.95):  
    alpha = 1 - gamma # уровень значимости
    t_crit = -t.ppf(alpha/2, k) # t-критическое
    eps = t_crit * x_std/(n ** 0.5) # погрешность
    lower_bound = x_mean - eps # левая (нижняя) граница
    upper_bound = x_mean + eps # правая (верхняя) граница
    # возвращаем кортеж из границ интервала
    return lower_bound, upper_bound

# cтроим доверительный интервал для среднего при неизвестном СКО
lower_bound, upper_bound = t_mean_conf_interval(x_mean, x_std, n)

# выводим результат
print('Доверительный интервал: {}'.format((round(lower_bound), round(upper_bound))))

# Доверительный интервал: (1778, 2222)

Доверительный интервал: (1778, 2222)


Итак, мы на 95 % уверены, что истинные траты покупателей в нашем магазине лежат в интервале от 1 778 до 2 222 рублей.


Примечание

Важно отметить, что при достаточной выборке (примерно более 30 наблюдений) распределение Стьюдента очень похоже на стандартное нормальное распределение. В этом случае можно использовать первую формулу (z-критическое) даже при неизвестном стандартном отклонении. Однако используя вторую формулу при оценке среднего, вы не ошибётесь.

Ещё один важный факт состоит в том, что обе формулы работают только при соблюдении двух условий:

Выборка должна быть случайной.
Выборка должна быть достаточно велика. Но есть исключение: если выборка маленькая, но сама величина распределена нормально, то выборочное среднее тоже будет распределено нормально. В таком случае мы можем пользоваться формулой доверительного интервала.

### Задание 6.6

0 points possible (ungraded)

Чиновника интересует сумма подоходного налога, выплачиваемого домохозяйствами за год. На случайной выборке из 25 домохозяйств был посчитан средний размер выплаченного налога, и он составил $ 3 540. 

Также известно выборочное отклонение величины подоходного налога, равное $ 1 150.

Необходимо найти 90 %-ый доверительный интервал для истинного среднего значения налоговых выплат за год. Используйте формулу с t-критическим.

Укажите границы построенного доверительного интервала для истинного среднего. Ответ округлите до целого.

Нижняя граница:
3146

Верхняя граница:
3934

In [20]:
n = 25 # размер выборки
k = n - 1 # число степеней свободы
x_mean = 3540 # выборочное среднее
x_std = 1150 # выборочное стандартное отклонение
gamma = 0.90 # уровень надёжности
alpha = 1 - gamma # уровень значимости

# Вычисляем t-критическое

t_critical = t.ppf(1 - alpha/2, k)
print(f"t-критическое: {t_critical:.2f}")

# cтроим доверительный интервал для среднего при неизвестном СКО
lower_bound, upper_bound = t_mean_conf_interval(x_mean, x_std, n, gamma)

# выводим результат
print('Доверительный интервал: {}'.format((round(lower_bound), round(upper_bound))))

t-критическое: 1.71
Доверительный интервал: (3146, 3934)


## ДОВЕРИТЕЛЬНЫЙ ИНТЕРВАЛ ДЛЯ ПРОПОРЦИИ

Настало время вернуться к нашей задаче об анализе конверсии в двух группах A/B-тестирования.

Давайте ещё раз вспомним, что такое конверсия. 

Конверсия — это доля пользователей, совершивших целевое действие, по отношению к общей массе пользователей. 

Ранее мы уже выяснили, что признак того, осуществлена конверсия пользователем или нет — это случайная величина, имеющая распределение Бернулли. Такая случайная величина имеет два возможных исхода: 

успех (1) — конверсия осуществлена (пользователь совершил целевое действие);
неуспех (0) — пользователь не совершил целевое действие. 
То есть конверсия — это вероятность того, что пользователь совершит целевое действие (вероятность успеха традиционно обозначается как p):

$p=conv= \frac{N^{+}}{N}$

 — $N^{+}$ - количество успехов (количество пользователей, осуществивших конверсию);
 — $N$ - общее количество исходов (количество пользователей);
 — $conv$ - конверсия.

Примечание. При расчётах конверсии обычно долю ещё умножают на 100 %.

В контексте задачи A/B-тестирования нам бы хотелось знать истинную конверсию (а не ту, которая подсчитана по выборке) в контрольной и тестовой группе, а также оценить истинную разницу между конверсиями. 

Если подумать, то на самом деле мы решаем задачу оценки истинной пропорции генеральной совокупности по выборке. А значит,  нужно научиться строить доверительный интервал для пропорции.

Давайте вспомним, что истинное среднее (математическое ожидание) случайной величины, распределённой по закону Бернулли, равно $\mu=p$, а истинное стандартное отклонение $\sigma_p = \sqrt{p(1-p)}$. 

Истинную вероятность успеха $p$ , мы, конечно же, не знаем, так как нам неизвестна вся генеральная совокупность (всех возможных пользователей). Вместо неё мы используем выборочную пропорцию (обозначим её как $X_p$) — долю совершивших конверсию пользователей, то есть выборочную конверсию. 

Заметим, что истинная пропорция $p$ полностью совпадает с истинным средним $\mu$. Если немного поколдовать с формулой доверительного интервала для истинного среднего, то можно вывести формулу доверительного интервала для истинной пропорции (истинной конверсии):

$p=\mu=X_{p} \pm z_{крит} \times \sqrt{\frac{X_{p}(1-X_{p})}{n}}$

Данная формула позволяет построить доверительный интервал для истинной конверсии в контрольной (А) и тестовой (B) группах.

Если сравнить со структурой доверительного интервала, введённой ранее, то можно заметить, что за выборочную оценку отвечает выборочная пропорция $\bar{x}=X_{p}$, а за погрешность интервала отвечает выражение $E=z_{крит} \times  \sqrt{\frac{X_{p}(1-X_{p})}{n}}$.

Примечание. Если размер выборки < 30, то вместо Z-критического значения используется t-критическое значение.
Но в задачах A/B-тестирования, как правило, такой размер выборки не используется. Чаще всего мы имеем дело с сотнями или тысячами наблюдений.

Пора приступать к реализации на Python. Будем работать с данными о посетителях веб-сайта из прошлого юнита:

In [22]:
ab_data = pd.read_csv('data/ab_data.zip')
ab_data.head()

,user_id,timestamp,group,converted
0,851104,2017-01-21,A,0
1,804228,2017-01-12,A,0
2,661590,2017-01-11,B,0
3,853541,2017-01-08,B,0
4,864975,2017-01-21,A,1


Для удобства создадим отдельные таблицы для каждой из групп пользователей:

In [23]:
# фильтруем данные группы А
a_data = ab_data[ab_data['group'] == 'A']
# фильтруем данные группы B
b_data = ab_data[ab_data['group'] == 'B']

Давайте на этот раз оформим расчёт в виде функции: 
объявим функцию proportion_conf_interval(). 
У данной функции будет три аргумента: 

x_p — выборочная пропорция, 

n — размер выборки и 

gamma — уровень надёжности (по умолчанию он равен 0.95). 

Функция будет возвращать кортеж из вычисленных границ доверительного интервала, умноженных на 100 % и округлённых до второго знака после запятой.

In [24]:
def proportion_conf_interval(x_p, n, gamma=0.95):  
    alpha = 1 - gamma # уровень значимости
    z_crit = -norm.ppf(alpha/2) # z-критическое
    eps = z_crit * (x_p * (1 - x_p) / n) ** 0.5 # погрешность
    lower_bound = x_p - eps # левая (нижняя) граница
    upper_bound = x_p + eps # правая (верхняя) граница
    # возвращаем кортеж из границ интервала
    return lower_bound, upper_bound

Теперь применим нашу функцию к данным группы А и группы B, результат умножим на 100 % и округлим до сотых:

In [25]:
# строим доверительный интервал для конверсии в группе А
lower_bound_a, upper_bound_a  = proportion_conf_interval(
    x_p=a_data['converted'].mean(), # выборочная пропорция
    n=a_data['user_id'].count() # размер выборки
)

# строим доверительный интервал для конверсии в группе B
lower_bound_b, upper_bound_b  = proportion_conf_interval(
    x_p=b_data['converted'].mean(), # выборочная пропорция
    n=b_data['user_id'].count() # размер выборки
)

# выводим результат
print('Доверительный интервал для конверсии в группе А: {}'.format((round(lower_bound_a * 100, 2), round(upper_bound_a * 100, 2))))
print('Доверительный интервал для конверсии в группе B: {}'.format((round(lower_bound_b * 100, 2), round(upper_bound_b * 100, 2))))

# Доверительный интервал для конверсии группы А: (11.86, 12.19)
# Доверительный интервал для конверсии группы B: (11.7, 12.03)

Доверительный интервал для конверсии в группе А: (11.86, 12.19)
Доверительный интервал для конверсии в группе B: (11.7, 12.03)


Итак, можно сделать вывод, что с надёжностью в 95 % конверсия группы А лежит в интервале от 11.86 % до 12.19 %, а конверсия группы B лежит в интервале от 11.7 % до 12.03 %.

Вычисление доверительного интервала можно было реализовать с помощью библиотечных инструментов, например, функцией proportion.proportion_confint() из модуля stats библиотеки statsmodels. Функция принимает на вход следующие аргументы:

count — количество успехов (в нашем случае это суммарное количество пользователей, выполнивших целевое действие);
nobs — количество наблюдений;
alpha — уровень значимости.
Пример вызова функции:

In [26]:
from statsmodels.stats import proportion

# вычисляем доверительный интервал для конверсии в группе A с уровнем доверия 0.95
lower_bound_a, upper_bound_a = proportion.proportion_confint(
    count=a_data['converted'].sum(),
    nobs=a_data['user_id'].count(),
    alpha=0.05
)

# вычисляем доверительный интервал для конверсии в группе B с уровнем доверия 0.95
lower_bound_b, upper_bound_b = proportion.proportion_confint(
    count=b_data['converted'].sum(),
    nobs=b_data['user_id'].count(),
    alpha=0.05
)
# выводим результат
print('Доверительный интервал для конверсии в группе А: {}'.format((round(lower_bound_a * 100, 2), round(upper_bound_a * 100, 2))))
print('Доверительный интервал для конверсии в группе B: {}'.format((round(lower_bound_b * 100, 2), round(upper_bound_b * 100, 2))))
# Доверительный интервал для конверсии в группе А: (11.86, 12.19)
# Доверительный интервал для конверсии в группе B: (11.7, 12.03)

Доверительный интервал для конверсии в группе А: (11.86, 12.19)
Доверительный интервал для конверсии в группе B: (11.7, 12.03)


Что значит такое пересечение интервалов для нас? Это значит, что существует вероятность, что истинное значение конверсии в обеих группах может совпасть. То есть мы не можем однозначно сказать, есть ли разница в конверсиях. Можно даже рассчитать эту вероятность, но лучше пойти другим путём.

Лучше построить доверительный интервал разницы пропорций (конверсий).

Не будем уходить в статистические дебри, просто приведём формулу для расчёта доверительного интервала разницы конверсий в группах. Запоминать её, как и считать руками, не нужно, мы реализуем расчёты с помощью Python. Однако важно понимать её составляющие:

$$\Delta p = \Delta X_{p} \pm z_{крит} \times \sqrt{\frac{X_{p_a}(1-X_{p_a})}{n_a}+\frac{X_{p_b}(1-X_{p_b})}{n_b}}$$

индексы a и b обозначают принадлежность параметра группе А и B соответственно;

 — $\Delta p = p_{b}-p_{a}$ — истинная разница конверсий групп B и A;

 — $\Delta X_{p} = X_{p_b}-X_{p_a}$ — выборочная разница конверсий групп B и A.

 Таким образом, в центре доверительного интервала находится $\Delta X_{p}$  — это выборочная оценка, — а границы интервала задаются погрешностью $E= z_{крит} \times \sqrt{\frac{X_{p_a}(1-X_{p_a})}{n_a}+\frac{X_{p_b}(1-X_{p_b})}{n_b}}$.

 

Допустим, мы построим такой доверительный интервал. Как тогда на его основе сделать выводы и принять решение? Возможны три случая:

- Истинная разница в конверсии $\Delta p = p_{b}-p_{a}$ положительная. В таком случае говорят, что вариант B более эффективен по метрике конверсии, чем вариант А с заданной надёжностью.

- Истинная разница в конверсии $\Delta p = p_{b}-p_{a}$ отрицательная. В таком случае говорят, что вариант A более эффективен по метрике конверсии, чем вариант B с заданной надёжностью.

- Интервал охватывает точку 0. Левая граница доверительного интервала отрицательная, а правая — положительная. То есть истинная разница в конверсии $\Delta p = p_{b}-p_{a}$ может быть как положительной, так и отрицательной. Тогда это будет значить, что варианты А и B равнозначны по метрике конверсии. 


Создадим функцию diff_proportion_conf_interval() со следующими аргументами: x_p — список из выборочных пропорций для групп А и B соответственно,  n — список из размеров выборки для групп А и B соответственно, и gamma — уровень надёжности (по умолчанию он равен 0.95). Функция будет возвращать кортеж из вычисленных границ доверительного интервала разницы конверсий, умноженных на 100 % и округлённых до второго знака после запятой.

In [27]:
def diff_proportion_conf_interval(x_p, n, gamma=0.95):
    alpha = 1 - gamma # уровень значимости
    diff = x_p[1] - x_p[0] # выборочная разница конверсий групп B и A
    z_crit = -norm.ppf(alpha/2) #z-критическое
    eps = z_crit * (x_p[0] * (1 - x_p[0])/n[0] + x_p[1] * (1 - x_p[1])/n[1]) ** 0.5 # погрешность
    lower_bound = diff - eps # левая (нижняя) граница
    upper_bound = diff + eps # правая (верхняя) граница
    # возвращаем кортеж из  границ интервала
    return lower_bound, upper_bound

Вызовем функцию и построим интересующий нас доверительный интервал:

In [28]:
# размеры выборок групп А и B
n = [a_data['user_id'].count(), b_data['user_id'].count()]
# выборочная пропорция групп A и B
x_p = [a_data['converted'].mean(), b_data['converted'].mean()]


# строим доверительный интервал для разности пропорций
lower_bound, upper_bound = diff_proportion_conf_interval(x_p=x_p, n=n)


# выводим результат
print('Доверительный интервал для разности конверсий: {}'.format((round(lower_bound*100, 2), round(upper_bound*100, 2))))

Доверительный интервал для разности конверсий: (-0.39, 0.08)


Итак, что можно сказать теперь? Построенный доверительный интервал с 95 %-ой надёжностью говорит нам, что разница в конверсии между вариантом B и вариантом А лежит в интервале от -0.39 % до 0.08 %. 

То есть разница в конверсии с 95 %-ой надёжностью может быть как положительной, так и отрицательной. Мы имеем дело с третьим случаем — интервал охватывает точку 0.

Отсюда можно сделать вывод, что статистической разницы в конверсии между вариантами А и B не существует, то есть они являются равнозначными по данной метрике.

В заключение хотелось бы отметить, что доверительный интервал для пропорции применяется не только для оценки конверсии и разницы конверсий в группах!

Под истинной пропорцией может пониматься доля успешно угаданных моделью объектов (accuracy), доля выздоровевших пациентов или доля ушедших клиентов (отток). То есть такой доверительный интервал может быть применён в любой задаче A/B-тестирования, где признак, по которому считается метрика, бинарный (имеет два возможных исхода — «успех» и «неуспех»). А это очень большой спектр задач!

Таким образом, доверительные интервалы — это ещё один вспомогательный инструмент проведения A/B-тестирования, который позволяет с определённой заранее надёжностью статистически сравнить варианты продукта А и B по заданной метрике. 

Обычно доверительные интервалы используются в комплексе со статистическими тестами. Статистический тест показывает, значимы ли наши результаты, а доверительный интервал показывает пределы изменения метрик и их разницы в группах. 

### Задание 6.9

3 points possible (graded)

Чтобы оценить удовлетворённость клиентов сети отелей, менеджер взял случайную выборку из 189 комментариев, оставленных посетителями на онлайн-платформе по бронированию, и провёл анализ их содержания. Выяснилось, что 132 комментария были положительными, остальные — негативными.

Давайте поможем менеджеру ответить на несколько вопросов!

Чему равна выборочная пропорция положительных комментариев? Укажите с точностью до трёх знаков после разделительной точки.

  нет ответа 
Найдите 90 % доверительный интервал для истинной пропорции положительных комментариев. Ответ приведите в процентах и округлите до двух знаков после запятой. При расчёте используйте z-критическое.


Нижняя граница:

Верхняя граница:

Подсказка (1 из 2):
Выборочная пропорция положительных комментариев — это число положительных комментариев, поделённое на число всех комментариев.

Подсказка (2 из 2):
После того как вы нашли выборочную пропорцию x_p, вам останется только подставить размер выборки n и полученную x_p в функцию proportions_conf_interval(), которую мы создали ранее. Не забудьте указать параметр gamma=0.9.

### Решение Задания 6.9

Расчет выборочной пропорции и доверительного интервала для отзывов.

In [29]:
import scipy.stats as st
import math

n = 189
positive = 132

# Выборочная пропорция
p_hat = positive / n
print(f"Выборочная пропорция: {p_hat:.3f}")

# 90% Доверительный интервал
gamma = 0.90
alpha = 1 - gamma
z_crit = abs(st.norm.ppf(alpha/2)) # z-критическое

margin_of_error = z_crit * math.sqrt((p_hat * (1 - p_hat)) / n)

lower = (p_hat - margin_of_error) * 100
upper = (p_hat + margin_of_error) * 100

print(f"90% Доверительный интервал: [{lower:.2f}%, {upper:.2f}%]")


Выборочная пропорция: 0.698
90% Доверительный интервал: [64.35%, 75.33%]


### Задание 6.10
3 points possible (graded)
Проводится A/B-тестирование вариантов отображения страницы корзины на сайте интернет-магазина. Есть два варианта отображения страницы: с товарными рекомендациями «Хиты продаж» и персональными, подобранными под интерес конкретно этого пользователя. Метрикой является конверсия действия «добавить в корзину товар из рекомендаций». Были собраны следующие данные:

Группа | Число добавленных в корзину товаров | Размер выборки
--- | --- | ---
A(Персональные скидки) | 45 | 1000
B(Хиты продаж) | 50 | 1000

Постройте 95 %-ый доверительный интервал разности полученных конверсий. Ответ приведите в процентах и округлите границы интервала до второго знака после запятой. При расчёте используйте z-критическое.

Нижняя граница:


Верхняя граница:


Какой вывод можно сделать на основе полученного доверительного интервала?


- вариант А является более предпочтительным варианту B по показателю конверсии
- вариант B является более предпочтительным варианту А по показателю конверсии
- варианты А и B равнозначны по показателю конверсии


Подсказка (1 из 2): Различные ответы могут получиться в зависимости от того, как берётся разница групп (из конверсии группы A вычитается конверсия группы B или наоборот).
Подсказка (2 из 2): Рассчитайте выборочные конверсии как число добавленных в корзину товаров/размер выборки в каждой группе. Вызовите функцию diff_proportions_conf_interval(), описанную ранее, передав в неё размеры выборок и выборочные пропорции в виде списков.

### Решение Задания 6.10

Доверительный интервал разности конверсий (Персональные скидки vs Хиты продаж).

In [30]:
from scipy.stats import norm
import math

n_a = 1000
x_a = 45
p_a = x_a / n_a

n_b = 1000
x_b = 50
p_b = x_b / n_b

def diff_proportions_conf_interval(n, p, gamma=0.95):
    alpha = 1 - gamma
    diff = p[1] - p[0]
    z_crit = abs(norm.ppf(alpha/2))
    eps = z_crit * math.sqrt(p[0]*(1-p[0])/n[0] + p[1]*(1-p[1])/n[1])
    return (diff - eps)*100, (diff + eps)*100

lower, upper = diff_proportions_conf_interval([n_a, n_b], [p_a, p_b])

print(f"95% Доверительный интервал разности (B-A): [{lower:.2f}%, {upper:.2f}%]")

if lower <= 0 <= upper:
    print("Вывод: Варианты A и B равнозначны по показателю конверсии.")
else:
    if lower > 0:
        print("Вывод: Вариант B лучше.")
    else:
        print("Вывод: Вариант A лучше.")


95% Доверительный интервал разности (B-A): [-1.36%, 2.36%]
Вывод: Варианты A и B равнозначны по показателю конверсии.
